In [1]:
#-----------------------------------------------------------------------------------------------------
# BERT와 sLLM 모델을 이용한 질의응답 서비스 구축 예
# - 여기서는 문서들을 전처리하고, 임베딩 하는 과정임.
#
# 질의 응답 시스템 과정
# 문서들 전처리 : 
#    단락별루 분할(\n\n) - 불용어 제거 -문장별루 분할.
# 임베딩 : 
#    kpf-sbert-v1.1로  문장 평균 임베딩벡터 구함 - es에 문장별루 단락text와 평균벡터 저장.
# 프롬프트생성 및 입력 : 
#   검색어 입력(회사:과장일때 휴가 일수는 얼마?)-bert로 임베딩 검색(*스코어가 0.6이상인 경우 체택)-sLLM에 검색된 단락 text를 문맥으로 해서 prompt 구성
#   sLLM에 prompt 입력-응답 결과 출력
#-----------------------------------------------------------------------------------------------------

import os
import random
import numpy as np
import pandas as pd
import time
import random

import sys
sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging, getListOfFiles

SEED = 111
seed_everything(SEED)
DEVICE = GPU_info() # GPU 혹은 CPU
LOGGER = mlogging(loggername="sllm-test", logfilename='../../log/sll-test.txt') # 로그

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
logfilepath:../../log/sll-test.txt_2023-11-01.log


In [2]:
# DATA_FOLDER에 파일들을 불러와서 DF로 만듬.
# -문서는 \n으로 구분해야 하며, 맨앞에는 title이 와야 하고, \n 다음 문단들이 와야함. 문단들은 \n 구분됨.
# -예: 회사 개요\n{회사내용}\n{제품구성}

# 파일이 여러개인 경우 폴더 지정
DATA_FOLDER = '../data11/mpower_doc/사규개정-out-renew/'

files = getListOfFiles(DATA_FOLDER)
assert len(files) > 0 # files가 0이면 assert 발생
print('*file_count: {}, file_list:{}'.format(len(files), files[0:5]))

# 파일 경로를 지정하면됨.
#files = ["../data11/mpower_doc/신입사원교육.txt",]
docid = 0  # **카운터가 문서에 uid가 되므로, 유일무이한 값므로 지정할것.
count = 0

titles = []
contextids = []
contexts = []

for idx, file_path in enumerate(files):
    if '.ipynb_checkpoints' not in file_path:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = f.read()
            #print(data)
            data_list = data.split('\n\n')  # '\n\n' 구분으로 다락 구분
            docid += 1
            count = 0
            # titles, contextids 임의로 구해서 데이터 프레임 만듬.
            for i in range(len(data_list)):
                if i > 0 and data_list[i]:
                    count += 1
                    titles.append(data_list[0].strip())  # 문서 제목은 맨처음 문장이고 \n\n로 구분되어야 함.
                    contextids.append(f'{docid}-{count}')
                    contexts.append(data_list[0].strip() + '\n' + data_list[i].strip()) # 두번째는 문서 제목+문서내용 합처서 문장만듬.
 
# 데이터 프레임으로 만듬.
df_contexts = pd.DataFrame((zip(contexts, titles, contextids)), columns = ['context','question', 'contextid'])     

print(f'*문단 총 계수:{len(df_contexts)}\n')

*file_count: 46, file_list:['../data11/mpower_doc/사규개정-out-renew/위임전결규정.txt', '../data11/mpower_doc/사규개정-out-renew/유형자산 관리 지침.txt', '../data11/mpower_doc/사규개정-out-renew/의료비 지원 규정.txt', '../data11/mpower_doc/사규개정-out-renew/이사회 운영규정.txt', '../data11/mpower_doc/사규개정-out-renew/인사 규정.txt']
*문단 총 계수:977



In [3]:
# JSON 파일로 저장해둠.
import json
examples = []

for context, title, contextid in zip(contexts, titles, contextids):
    doc = {}
    doc['context'] = context  # 문단
    doc['title'] = title      # 제목
    doc['contextid'] = contextid # 문서 id
                        
    examples.append(doc)
                        
docs ={}
docs['text'] = examples

#json 파일로 저장.
# JSON 파일을 엽니다.
with open("./data/moco_context.json", "w") as f:
    # 리스트를 JSON으로 변환합니다.
    json.dump(docs, f, indent=4)

In [4]:
# 문장들로 분리
from myutils import get_sentences
doc_sentences = get_sentences(df=df_contexts, remove_sentnece_len=1, remove_duplication=False)

print(f'len:{len(doc_sentences)}, 1: {doc_sentences[1]}')

  0%|          | 0/977 [00:00<?, ?it/s][Kss]: Oh! You have konlpy.tag.Mecab in your environment. Kss will take this as a backend! :D

100%|██████████| 977/977 [00:05<00:00, 166.28it/s]

*[get_sentences] 문장처리=>len:3, time:5.8805
*[get_sentences] 문장 길이=>평균:5.60388945752303 / MAX: 23 / MIN: 1

len:977, 1: ['위임전결규정', '목적', '이 규정은 회사 업무 집행에 관련된 권한의 한계와 책임의 소재를 분명히 하여 업무수행의 신속과 효율 제고를 도모함을 목적으로 한다.']


In [5]:
# 인덱스 추가
from tqdm.notebook import tqdm
from myutils import embed_text, bi_encoder, mpower_index_batch
from myutils import create_index

# ES 관련
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import bulk
    
# 조건에 맞게 임베딩 처리하는 함수 
def embedding(paragraphs:list)->list:
    # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
    embeddings = embed_text(model=BI_ENCODER1, paragraphs=paragraphs, return_tensor=False).astype(FLOAT_TYPE)    
    return embeddings

#---------------------------------------------------------------------------
#문단에 문장들의 임베딩을 구하여 각각 클러스터링 처리함.
#---------------------------------------------------------------------------
def index_data(es, df_contexts, doc_sentences:list):
    #클러스터링 계수는 문단의 계수보다는 커야 함. 
    #assert num_clusters <= len(doc_sentences), f"num_clusters:{num_clusters} > len(doc_sentences):{len(doc_sentences)}"
    #-------------------------------------------------------------
    # 각 문단의 문장들에 벡터를 구하고 리스트에 저장해 둠.
    start = time.time()
    cluster_list = []

    rfile_names = df_contexts['contextid'].values.tolist()
    rfile_texts = df_contexts['context'].values.tolist()

    if OUT_DIMENSION == 0:
        dimension = 768
    else:
        dimension = 128

    clustering_num = NUM_CLUSTERS
        
    docs = []
    count = 0
    for i, sentences in enumerate(tqdm(doc_sentences)):
        embeddings = embedding(sentences)
        if i < 3:
            print(f'[{i}] sentences-------------------')
            if len(sentences) > 5:
                print(sentences[:5])
            else:
                print(sentences)
                
            LOGGER.info(f'*[index_data] embeddings.shape: {embeddings.shape}')
            print()
        
        #----------------------------------------------------------------
        multiple = 1
        
        # [bong][2023-04-28] 임베딩 출력 계수에 따라 클러스터링 계수를 달리함.
        if NUM_CLUSTERS_VARIABLE == True:
            embeddings_len = embeddings.shape[0]
            if embeddings_len > 2000:
                multiple = 6
            elif embeddings_len > 1000:
                multiple = 5 # 5배
            elif embeddings_len > 600:
                multiple = 4 # 4배
            elif embeddings_len > 300:
                multiple = 3 # 3배
            elif embeddings_len > 100:
                multiple = 2 # 2배
        #----------------------------------------------------------------
        
        # 0=문장클러스터링 임베딩
        if EMBEDDING_METHOD == 0:
            if CLUSTRING_MODE == "kmeans":
                # 각 문단에 분할한 문장들의 임베딩 값을 입력해서 클러스터링 하고 평균값을 구함.
                # [bong][2023-04-28] 문장이 많은 경우에는 클러스터링 계수를 2,3배수로 함
                emb = clustering_embedding(embeddings = embeddings, outmode=OUTMODE, num_clusters=(clustering_num*multiple), seed=SEED).astype(FLOAT_TYPE) 
            else:
                emb = kmedoids_clustering_embedding(embeddings = embeddings, outmode=OUTMODE, num_clusters=(clustering_num*multiple), seed=SEED).astype(FLOAT_TYPE) 
            
        # 1= 문장평균임베딩
        elif EMBEDDING_METHOD == 1:
            # 문장들에 대해 임베딩 값을 구하고 평균 구함.
            arr = np.array(embeddings).astype(FLOAT_TYPE)
            emb = arr.mean(axis=0).reshape(1,-1) #(128,) 배열을 (1,128) 형태로 만들기 위해 reshape 해줌
            clustering_num = 1  # 평균값일때는 NUM_CLUSTERS=1로 해줌.
        # 2=문장임베딩
        else:
            emb = embeddings

        if i < 3:
            LOGGER.info(f'*[index_data] cluster emb.shape: {emb.shape}')
            print()
        
        #--------------------------------------------------- 
        # docs에 저장 
        #  [bong][2023-04-28] 여러개 벡터인 경우에는 벡터를 10개씩 분리해서 여러개 docs를 만듬.
        for j in range(multiple):
            count += 1
            doc = {}                                #dict 선언
            doc['rfile_name'] = rfile_names[i]      # contextid 담음
            doc['rfile_text'] = rfile_texts[i]      # text 담음.
            doc['dense_vectors'] = emb[j * clustering_num : (j+1) * clustering_num] # emb 담음.
            docs.append(doc)
        #---------------------------------------------------    

            if count % BATCH_SIZE == 0:
                mpower_index_batch(es, ES_INDEX_NAME, docs, vector_len=clustering_num, dim_size=dimension)
                docs = []
                LOGGER.info("[index_data](1) Indexed {} documents.".format(count))

    if docs:
        mpower_index_batch(es, ES_INDEX_NAME, docs, vector_len=clustering_num, dim_size=dimension)
        LOGGER.info("[index_data](2) Indexed {} documents.".format(count))   

    es.indices.refresh(index=ES_INDEX_NAME)

    LOGGER.info(f'*인덱싱 시간 : {time.time()-start:.4f}\n')
    print()
#---------------------------------------------------------------------------


# param--------------------------------------------------------------------
OUT_DIMENSION = 0   # 128 혹은 768이면 0입력
EMBEDDING_METHOD=1  # 0=클러스터링 임베딩, 1=문장평균임베딩, 2=문장임베딩
NUM_CLUSTERS=10     # 클러스터링 임베딩일때 클러스터링 수 
NUM_CLUSTERS_VARIABLE=False # 클러스터링 임베딩일때 클러스터링을 문장계수마다 다르계할지.


MODEL_PATH = '../data11/model/kpf-sbert-v1.1'
POLLING_MODE = 'mean' # 폴링모드 
FLOAT_TYPE = 'float16' # float32 혹은 float16

# ES 접속
ES_URL = 'http://10.10.4.10:9200/'               # es 접속 주소
ES_INDEX_NAME = 'qaindex'          # 생성혹은 추가할 인덱스 명
ES_INDEX_FILE = './data/mpower10u_768d_1.json'   # 인덱스 구조 파일경로
BATCH_SIZE=20       # ES 배치 사이즈
CREATE_INDEX = True # True이면 기존에 인덱스가 있다면 제거하고 다시 생성.
# param--------------------------------------------------------------------

es = Elasticsearch(ES_URL)
create_index(es, ES_INDEX_FILE, ES_INDEX_NAME, create=CREATE_INDEX)

# 임베딩 모델 로딩
WORD_EMBDDING_MODEL1, BI_ENCODER1 = bi_encoder(model_path=MODEL_PATH, max_seq_len=512, do_lower_case=True, 
                                               pooling_mode=POLLING_MODE, out_dimension=OUT_DIMENSION, device=DEVICE)

print(BI_ENCODER1)
print()
try:
    index_data(es, df_contexts, doc_sentences)
except Exception as e:
    error = f'index_data fail'
    msg = f'{error}=>{e}'
    LOGGER.error(f'/embed/es {msg}')

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


1:{			"settings": {
				"number_of_shards": 2,
				"number_of_replicas": 1,
				"index": {
					"merge": {
						"scheduler": {
							"max_thread_count": 1
						}
					}
				},
				"analysis": {
					"analyzer": {
						"mpower10u_korean_analyzer": {
							"type": "custom",
							"tokenizer": "mpower10u_korean_tokenizer",
							"filter": [
								"lowercase",
								"stop",
								"nori_readingform",
								"mpower10u_korean_pos"
							]
						}
					},
					"tokenizer": {
						"mpower10u_korean_tokenizer": {
							"type": "nori_tokenizer",
							"decompound_mode": "discard"
						}
					},
					"filter": {
						"mpower10u_korean_pos": {
							"type": "nori_part_of_speech",
							"stoptags": [
								"J", "E", "XSA", "XSN", "XSV"
							]
						}
					}
				}
			},
			"mappings": {
				"_source": {
					"enabled": true
				},
				"dynamic":"strict",
				"properties": {
					"rfile_name": {
						"type": "text",
						"store": true
					},
					"rfile_text": {
						"ty

  0%|          | 0/977 [00:00<?, ?it/s]

2023-11-01 16:12:43,680 - sllm-test - INFO - *[index_data] embeddings.shape: (3, 768)
[Kss]: *[index_data] embeddings.shape: (3, 768)
2023-11-01 16:12:43,684 - sllm-test - INFO - *[index_data] cluster emb.shape: (1, 768)
[Kss]: *[index_data] cluster emb.shape: (1, 768)
2023-11-01 16:12:43,721 - sllm-test - INFO - *[index_data] embeddings.shape: (3, 768)
[Kss]: *[index_data] embeddings.shape: (3, 768)
2023-11-01 16:12:43,723 - sllm-test - INFO - *[index_data] cluster emb.shape: (1, 768)
[Kss]: *[index_data] cluster emb.shape: (1, 768)
2023-11-01 16:12:43,764 - sllm-test - INFO - *[index_data] embeddings.shape: (4, 768)
[Kss]: *[index_data] embeddings.shape: (4, 768)
2023-11-01 16:12:43,768 - sllm-test - INFO - *[index_data] cluster emb.shape: (1, 768)
[Kss]: *[index_data] cluster emb.shape: (1, 768)


[0] sentences-------------------
['위임전결규정', '적용범위', '주식회사 모코엠시스(이하 회사라 한다)의 업무수행에 관련된 모든 권한과 책임은 다른 사규로 정하지 않는 한 이 규정을 따르고 이 규정에 정하지 않는 권한과 책임은 대표이사가 정한다.']


[1] sentences-------------------
['위임전결규정', '목적', '이 규정은 회사 업무 집행에 관련된 권한의 한계와 책임의 소재를 분명히 하여 업무수행의 신속과 효율 제고를 도모함을 목적으로 한다.']


[2] sentences-------------------
['위임전결규정', '용어의 정의', '본부장은 회사의 직책상 사업본부를 맡고 있는 임원을 말한다.', '부서(팀)장은 직위에 관계없이 회사의 직책상 부서(팀)을 맡고 있는 책임자를 말한다.']




...

2023-11-01 16:13:15,260 - sllm-test - INFO - [index_data](1) Indexed 940 documents.
[Kss]: [index_data](1) Indexed 940 documents.


  0%|          | 0/20 [00:00<?, ?it/s]

2023-11-01 16:13:15,852 - sllm-test - INFO - [index_data](1) Indexed 960 documents.
[Kss]: [index_data](1) Indexed 960 documents.


  0%|          | 0/17 [00:00<?, ?it/s]

2023-11-01 16:13:16,411 - sllm-test - INFO - [index_data](2) Indexed 977 documents.
[Kss]: [index_data](2) Indexed 977 documents.
2023-11-01 16:13:16,454 - sllm-test - INFO - *인덱싱 시간 : 35.4102

[Kss]: *인덱싱 시간 : 35.4102

